In [1]:
import sys
!{sys.executable} -m pip install psycopg2
import psycopg2

#init crediential storage
userName = None
passWord = None

#init report vars
er_salaries = None
er_inv_cost = None
ir_orderNumber = None
ir_modelID = None
ir_customerID = None
cr_mNumber = None
cr_quantitySold = None
cr_customerID = None
rr_saleValue = None
rr_employeeID = None
rr_customerID = None

def get_priv(conn, userName):
    command = """SELECT priviledge FROM Employee WHERE firstName = '%s';""" % (userName)
    cur = conn.cursor()
    cur.execute(command)
    priv = cur.fetchone()
    cur.close()
    return priv

def admin_test(cur):
    
    #for cmd in create_roles():
        #cur.execute(cmd)
    #print("finished create roles")
    cur.execute(create_user("t_username", "t_password"))
    print("finished create user")
    for cmd in create_views():
        cur.execute(cmd)
    print("finished create views")
    for cmd in grant_privs():
        cur.execute(cmd)
    print("finished grant privs")
    for cmd in setup_tables():
        cur.execute(cmd)
    print("finished setup tables")
    for cmd in build_rev_report(1):
        cur.execute(cmd)
    print("finished rev report")
    for cmd in build_cust_report(1):
        cur.execute(cmd)
    print("finished cust report")
    for cmd in build_inv_report():
        cur.execute(cmd)
    print("finished inv report")
    for cmd in build_expense_report():
        cur.execute(cmd)
    print("finished cust report")
    for cmd in view_rev_report(1):
        cur.execute(cmd)
    print("finished view rev report")
    for cmd in view_cust_report(1):
        cur.execute(cmd)
    print("finished view cust report")
    view_inv_report()
    print("finished view inv report")
    for cmd in view_expense_report():
        cur.execute(cmd)
    print("finished view cust report")
    for cmd in create_customer(1,"testfirst", "testlast"):
        cur.execute(cmd)
    print("finished create customer")
    for cmd in create_order():
    
def get_commands(priv, instr, cur):
    aswitcher = {
            #"create_roles" : create_roles(),
            "create_user" : create_user(instr[1],instr[2]),
            "create_views": create_views(),
            "grant_privs" : grant_privs(),
            "setup_tables" : setup_tables(),
            "build_rev_report" : build_rev_report(instr[1]),
            "build_cust_report" : build_cust_report(instr[1]),
            "build_inv_report" : built_inv_report(),
            "build_expense_report" : build_expense_report(),
            "view_rev_report" : view_rev_report(),
            "view_cust_report" : view_cust_report(),
            "view_inv_report" : view_inv_report(),
            "view_expense_report" : view_expense_report(),
            "view_customer" : view_customer(),
            "create_customer" : create_customer(instr[1],instr[2],instr[3]),
            "create_order" : create_order(),
            "access_model" : access_model(instr[1]),
            "update_model" : update_model(instr[1], instr[2]),
            "access_inventory" : access_inventory(),
            "update_inventory" : update_inventory(instr[1], instr[2], instr[3], instr[4], instr[5]),
            "view_employees" : view_employees(),
            "access_employee" : access_employee(instr[1]),
            "update_employee" : update_employee(instr[1], instr[2], instr[3], instr[4]),
            "view_employee_sales" : view_employee_sales(instr[1]),
            "admin_test" : admin_test(cur)
        }

    sswitcher = {
            "view_sales_report" : view_sales_report(),
            "view_customer" : view_customer(),
            "create_customer" : create_customer(instr[1],instr[2],instr[3]),
            "update_customer" : update_customer(),
            "create_order" : create_order()
        }
        
    eswitcher = {
            "view_employees" : view_employees_limited(),
        }
        
    hswitcher = {
            "access_employee" : access_employee(instr[1]),
            "update_employee" : update_employee(instr[1], instr[2], instr[3], instr[4]),
            "view_employee_sales" : view_employee_sales(instr[1])     
        }
            
    privswitcher = {
            "Administrator" : aswitcher,
            "Sales" : sswitcher,
            "Engineer" : eswitcher,
            "HR" : hswitcher
        }
        
    return privswitcher[priv].get(instr[0], "Invalid Command\n")

def main():
    conn = None
    userName = input("Username: ")
    passWord = input("Password: ")
    
    try:
        # connect to PostgreSQL
        conn = psycopg2.connect(host="localhost",database="testdb", user=userName, password=passWord)
        cur = conn.cursor()
        
        #cur.execute("""CREATE USER administrator  PASSWORD 'admin';""")
        print("WELCOME!!")
        
        cur.execute("GRANT SELECT on Employee to administrato;")
        #cur.execute(create_user())
        
        #cmds = setup_tables()
        #for cmd in cmds:
        #    cur.execute(cmd)
        
        #cmds = create_views()
        #for cmd in cmds:
        #    cur.execute(cmd)
        
        #cmds = create_roles()
        #for cmd in cmds:
        #    cur.execute(cmd)
        
        #cmds = grant_privs()
        #for cmd in cmds:
        #    cur.execute(cmd)
        #print("finished running grant_privs")
        conn.commit()
        priv = get_priv(conn, userName)
        cur.close()
        conn.commit()
        
        stay = True
        while stay:
            instr = input("> ").lower().split()
            
            commands = get_commands(priv, instr, cur)
            
            cur = conn.cursor()
        
            for command in commands:
                cur.execute(command)
            
            cur.close()
            conn.commit()

    #Handle exceptions    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    #Ensure closed
    finally:
        if conn is not None:
            conn.close()
            
#def create_roles():
    #print("Running create_roles")
    #return ("""
    #    CREATE ROLE administrator with PASSWORD 'admin';
   #     ""","""
    #    CREATE ROLE sales_person;
     #   ""","""
    #    CREATE ROLE engineer;
     #   ""","""
     #   CREATE ROLE HR;
     #   """)
    
def create_user (username,password): #https://pythontic.com/database/mysql/create%20user   
    return ("""
        CREATE USER %s WITH PASSWORD '%s';
        """%username,password)

def create_views():
    print("Running create_views")
    return ("""
        CREATE engineerView as
            select firstName, lastName, priviledge
            from Employee;
        """)    

def grant_privs ():
    print("Running grant_privs")
    return ("""
        GRANT ADMIN to administrato;
        ""","""
        GRANT INSERT on employee to administrato with grant option;
        ""","""
        GRANT UPDATE, SELECT on customer to sales_person;
        ""","""
        GRANT INSERT on orders to sales_person;
        ""","""
        GRANT UPDATE, SELECT on model to engineer;
        ""","""
        GRANT UPDATE, SELECT on inventory to engineer;
        ""","""
        GRANT UPDATE, SELECT on employee to HR"""
        )
def setup_tables ():
    print("Running setup_tables")
    return ( """
        CREATE TYPE stENUM AS ENUM ('Hourly', 'Salary');
        ""","""
        CREATE TYPE pENUM AS ENUM ('Administrator', 'Sales', 'Engineer', 'HR');
        ""","""
        create table customer (
        customerId int,
        firstName varchar(20),
        lastName varchar(20),
        Primary key (customerID)
        );
        ""","""
        Create table model(
        mNumber int,
        salePrice numeric(8,2),
        Primary key (mNumber)
        );
        ""","""
        create table inventory(
        modelID int,
        cost numeric(8,2),
        leadTime interval,
        inStock int,
        category varchar(32),
        foreign key (modelID) references model(mNumber),
        primary key (modelID)
        );
        ""","""
        create table composes(
        mNumber    int,
        foreign key (mNumber) references model(mNumber),
        Primary key (mNumber)
        );
        ""","""
        Create table Employee (
        firstName varchar(32),
        lastName varchar(32),
        ssn int,
        employeeID int,
        Salary int,
        salaryType stENUM,
        userID int,
        priviledge pENUM,
        loginTime timestamp,
        logoutTime timestamp,
        Primary key (employeeID)
        );
        ""","""
        create table orders (
        orderNumber int,
        quantitySold int,
        saleValue numeric(8,2),
        Primary key (orderNumber),
        employeeID int,
        customerID int,
        mNumber int,
        Foreign key (employeeID) references Employee(employeeID),
        Foreign key (customerID) references customer(customerID),
        Foreign key (mNumber) references model(mNumber)
        );
        """
    )    

def build_rev_report(orderID):
    return "SELECT saleValue, employeeID, customerID from Orders;"
        
        #rr_saleValue = row[0]
        #rr_employeeID = row[1]
        #rr_customerID = row[2]

def build_cust_report(cID): 
        return "SELECT mNumber, SUM(quantitySold), customerID FROM order WHERE customerID = %s GROUP BY mNumber;"
        
        #cr_mNumber = row[0]
        #cr_quantitySold = row[1]
        #cr_customerID = row[2]

def build_inv_report():
    return "SELECT orderNumber, modelID, inStock FROM order;"
        
        #rows = cur.fetchall()
        #for row in rows:
            #ir_orderNumber = row[0]
            #ir_modelID = row[1]
            #ir_customerID = row[2]
            
def build_expense_report():
    #Expense report, employee showing salary, bonus expense and part cost
    #All expenses included in Employee.Salary_Type = Salary & Inventory.Cost
    return ("SELECT SUM(Salary) FROM Employee WHERE Salary_Type = 'Salary';", "SELECT SUM(Cost) FROM Inventory;")
        
        #er_salaries = cur.fetchone()
        #er_inv_cost = cur.fetchone()

def view_rev_report():
    if rr_employeeID is not None:
        print("Employee %s was responsible for $ %s in revenue from customer %s" % (rr_employeeID, rr_saleValue, rr_customerID))
    else:
        print("Revenue report not yet build, please contact admin for report generation")


def view_cust_report():
    if cr_customerID is not None:
        print("Customer %s purchased %s units of model %s" % (cr_customerID, cr_numSold, cr_mNumber ))
    else:
        print("Customer report not yet build, please contact admin for report generation")


def view_inv_report():
    if ir_orderNumber is not None:
        print("Order %s requires part %s of which there are %s in stock" % (ir_orderNumber, ir_modelID, ir_customerID ))
    else:
        print("Inventory report not yet build, please contact admin for report generation")


def view_expense_report():
    if er_salaries is not None:
        f"The annual cost of salaried employees is ${er_salaries} and the total cost of recorded inventory is {er_inv_cost}"
    else:
        print("Expense report not yet build, please contact admin for report generation")  


def view_customer(customer_id):
    return """SELECT * FROM customer
             WHERE customer_id = customer.customer_id;"""


def create_customer(customer_id, firstName, lastName):
    return """INSERT into customer VALUES(%d, %s, %s);"""%(customer_id, firstname, lastName)
    

def create_order(orderNumber, quantity, total, empID, custID, model_id):
    return """INSERT into orders (orderNumber, quantitySold, saleValue, employeeID, customerID, mNumber) 
             VALUES(%d, %d, %8.2f, %d, %d, %d);"""%(orderNumber, quantity, total, empID, custID, model_id) 
    

def access_model(model_id):
    return """SELECT salePrice FROM model
             WHERE model_id = model.mNumber;"""
    
def update_model(model_id, price):
    return """UPDATE model
             SET salePrice = %8.2f;"""%(price)
    
def access_inventory():
    return """SELECT * FROM inventory;"""

def update_inventory(mNumber, cost, leadtime, category, instock): 
    return """UPDATE inventory 
            SET cost = %s, leadtime = %s, category = %s, inStock = %s
            WHERE mNumber = %s"""%(cost,leadtime, category, instock, mNumber) 

def view_employees_limited():
    return """SELECT firstName, lastName, priviledge FROM engineerView;"""

def view_employees():
    return """SELECT * FROM employee;"""
    
def access_employee(employee_id):
    return """SELECT * FROM employee WHERE %d = employeeID;"""%(employee_id)
   
def update_employee(employee_id, SalaryType, Salary, priviledge):
    return """UPDATE employee
             SET Salary = %s, SalaryType = %s, 
             WHERE %d = employeeID;"""%(employee_id)
   
def view_employee_sales(employee_id): 
    return """SELECT SUM(saleValue) FROM orders WHERE %d = employeeID;"""%(employee_id)

In [ ]:
main()